# Building and evaluating transformer models and their optimal hyperparameters
#### Evaluate and then pick the one I like to implement in my app

In [ ]:
# next time I start a session, run without comments
from google.colab import drive
drive.mount('/content/drive')

# %cd /content/drive/MyDrive/FakeProfileDetection

ModuleNotFoundError: No module named 'google'

In [ ]:
#delete once complete
!git clone https://github.com/hannahishimwe/FakeProfileDetection.git /content/drive/MyDrive/FakeProfileDetection

In [ ]:
# %%capture
!pip install pandas transformers datasets torch transformers[torch]

In [ ]:
!pip install emoji

In [ ]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from FakeProfileDetection.dev.CleanDatasets import CleanDatasets
from datasets import Dataset



In [ ]:
PATH_TO_TRAINING_CSV = "./FakeProfileDetection/dev/csv/structured_train.csv"
PATH_TO_TEST_CSV = "./FakeProfileDetection/dev/csv/structured_test.csv"
MODEL_NAME = "FacebookAI/roberta-base"
TOKENIZER_MODEL_NAME = "roberta-base"
MODEL = RobertaForSequenceClassification
TOKENIZER = RobertaTokenizer



NameError: name 'RobertaForSequenceClassification' is not defined

In [ ]:

training_df = pd.read_csv(PATH_TO_TRAINING_CSV)
testing_df = pd.read_csv(PATH_TO_TEST_CSV)
pd.set_option('display.max_colwidth', None)
training_df.head(50)

columns_to_rename_dict = {"screen_name": "username", "account.type": "is_human"}
columns_to_drop_list = ["class_type"]
column_to_binary = "account.type"


training_cleaner = CleanDatasets(training_df, columns_to_rename_dict, columns_to_drop_list, column_to_binary)
clean_training_df = training_cleaner.clean_df()
testing_cleaner = CleanDatasets(testing_df, columns_to_rename_dict, columns_to_drop_list, column_to_binary)
clean_testing_df = testing_cleaner.clean_df()
train_dataset = Dataset.from_pandas(clean_training_df)
test_dataset = Dataset.from_pandas(clean_testing_df)
train_dataset

In [ ]:
# Tokenize the texts - replace 'text' in each column with vectors, attention mask and label so machines can work with it
tokenizer = RobertaTokenizer.from_pretrained(TOKENIZER_MODEL_NAME)

# Tokenize the texts
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

train_tokenized = train_dataset.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)
train_tokenized = train_tokenized.rename_column("is_human", "labels")
test_tokenized = test_tokenized.rename_column("is_human", "labels")

train_tokenized = train_tokenized.remove_columns(["username", "text"])
test_tokenized = test_tokenized.remove_columns(["username", "text"])

train_tokenized[0].keys()

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
#configurate the trainer
trainer = Trainer(
    model=MODEL.from_pretrained(MODEL_NAME),
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized
)
trainer.train()
trainer.save_model("/content/drive/MyDrive/roberta_round1")

# to load again
# model_path = "/content/drive/MyDrive/roberta_round1"
# model = RobertaForSequenceClassification.from_pretrained(model_path)

In [ ]:
eval_results = trainer.evaluate("./FakeProfileDetection/dev/csv/structured_validation.csv")
training_df = pd.read_csv(PATH_TO_TRAINING_CSV)
valid_cleaner = CleanDatasets(testing_df, columns_to_rename_dict, columns_to_drop_list, column_to_binary)
clean_testing_df = testing_cleaner.clean_df()
train_dataset = Dataset.from_pandas(clean_training_df)